In [ ]:
import pandas as pd
import re
import pickle
import os.path
import os
import io
from flask import Flask, flash, request, redirect, url_for

app = Flask(__name__)

imported = pd.read_csv('data/all_data_marked.csv')

with open('model.pic','rb') as f:
    model = pickle.load(f)
with open('les.pk','rb') as f:
    les = pickle.load(f)  

@app.route('/model/uplearn/<string:file_name>/')
def prolearn_model(file_name):
    intro_df = pd.read_html("data/" + file_name[1:-1])
    intro_df = intro_df[1]
    feach = preprocess("data/" + file_name[1:-1])
    feach = feature_eng(feach)
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(feach.drop('Адресат',axis=1), feach['Адресат'], test_size=0.15)
    model.fit(x_train,y_train)
    with open('model.pic','wb') as f:
        pickle.dump(model,f)
    return "Successfully learned"


@app.route('/find_name/<name>/')
def get_mails(name):
    res = imported[imported['Адресат'] == name]
    res.fillna('')
    if res.empty:
        return f'Sorry, "{name}" not founded'
    else: 
        return res.to_html()

@app.route('/model/prediction/<file_name>/')
def get_predictions(file_name):
    file = 'data/' + file_name[1:-1]
    file_name = file_name[1:-1]
    try:    
        data = pd.read_html(file)[1]
        head = pd.read_html(file)[0]
    except ValueError:
        return f'File {file} not found'
    ldf = preprocess(file)    
    ldf = feature_eng_out(ldf)
    pred = model.predict(ldf)
    nms = []
    for i in pred:
        nms.append(names[int(i)])
    data["Адресат"] = nms
    

    result_link = f'results/{file_name[:-4]}_predicted.xls'
    data.to_excel(f'results/{file_name[:-4]}_predicted.xls')

    return result_link


def preprocess(name):
    intro_df = pd.read_html(name)
    intro_df = intro_df[1]

    intro_df['Дата'] = intro_df['Исх. №Дата'].map(get_data)
    intro_df['Исх. №'] = intro_df['Исх. №Дата'].map(get_num)

    # Удаляем исходный столбик за ненадобностью
    intro_df.drop('Исх. №Дата', axis=1, inplace=True)

    # Применение вышеуказанных функций
    intro_df['Организация'] = intro_df['Автор'].map(get_org)
    intro_df['Автор'] = intro_df['Автор'].map(get_name)

    # Выделение вышеописаных атрибутов в отдельный DataFrame
    resulted_df = intro_df[['Адресат', 'Автор', 'Краткоесодержание', 'Примечание', 'Организация']]

    # Приведем слова к общему виду, поставив все слова в именительный падеж и удалив знаки препинания

    import pymorphy2 as mrf
    morph = mrf.MorphAnalyzer()

    # Функция преобразующая все слова в тексте в именительный падеж
    def txt_normolize(s):
        temp = s.split(' ')
        res = [morph.parse(temp[i])[0].normal_form.rstrip('-,') for i in range(len(temp))]
        res = [x for x in res if x]
        return res

    resulted_df['Краткоесодержание'] = resulted_df['Краткоесодержание'].map(txt_normolize)

    return resulted_df


def feature_eng(df):
    reeng_feach = pd.DataFrame()

    stat = {}
    for i in df['Автор']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    autor = []
    for i in df['Автор']:
        autor.append(stat.index(i))
    reeng_feach['Автор'] = autor

    stat = {}
    for i in df['Адресат']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    adres = []
    for i in df['Адресат']:
        adres.append(stat.index(i))
    reeng_feach['Адресат'] = adres

    stat = {}
    for i in df['Организация']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    org = []
    for i in df['Организация']:
        org.append(stat.index(i))
    reeng_feach['Организация'] = org

    dic = {}
    for i in df['Краткоесодержание']:
        for j in i:
            if j not in dic:
                dic[j] = 1
            else:
                dic[j] += 1

    def get_pop(m):
        t = 0
        temp = []
        for i in m:
            if dic[i] > t:
                temp.append(i)
                t = dic[i]
                if len(temp) >= 4:
                    temp = temp[1:]
        while len(temp) < 3:
            temp.append('')
        return temp

    sod = []
    for i in df['Краткоесодержание']:
        sod.append(get_pop(i))
    key1 = []
    key2 = []
    key3 = []
    for i in sod:
        key1.append(i[0])
        key2.append(i[1])
        key3.append(i[2])
    reeng_feach['Keyword1'] = key1
    reeng_feach['Keyword2'] = key2
    reeng_feach['Keyword3'] = key3

    stat = {}
    for i in reeng_feach['Keyword1']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    soderj = []
    for i in reeng_feach['Keyword1']:
        soderj.append(stat.index(i))
    reeng_feach['Keyword1'] = soderj

    stat = {}
    for i in reeng_feach['Keyword2']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    soderj = []
    for i in reeng_feach['Keyword2']:
        soderj.append(stat.index(i))
    reeng_feach['Keyword2'] = soderj

    stat = {}
    for i in reeng_feach['Keyword3']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    soderj = []
    for i in reeng_feach['Keyword3']:
        soderj.append(stat.index(i))
    reeng_feach['Keyword3'] = soderj
    return reeng_feach

def feature_eng_out(df):
    reeng_feach = pd.DataFrame()

    stat = {}
    for i in df['Автор']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    autor = []
    for i in df['Автор']:
        autor.append(stat.index(i))
    reeng_feach['Автор'] = autor

    stat = {}
    for i in df['Организация']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    org = []
    for i in df['Организация']:
        org.append(stat.index(i))
    reeng_feach['Организация'] = org

    dic = {}
    for i in df['Краткоесодержание']:
        for j in i:
            if j not in dic:
                dic[j] = 1
            else:
                dic[j] += 1

    def get_pop(m):
        t = 0
        temp = []
        for i in m:
            if dic[i] > t:
                temp.append(i)
                t = dic[i]
                if len(temp) >= 4:
                    temp = temp[1:]
        while len(temp) < 3:
            temp.append('')
        return temp

    sod = []
    for i in df['Краткоесодержание']:
        sod.append(get_pop(i))
    key1 = []
    key2 = []
    key3 = []
    for i in sod:
        key1.append(i[0])
        key2.append(i[1])
        key3.append(i[2])
    reeng_feach['Keyword1'] = key1
    reeng_feach['Keyword2'] = key2
    reeng_feach['Keyword3'] = key3

    stat = {}
    for i in reeng_feach['Keyword1']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    soderj = []
    for i in reeng_feach['Keyword1']:
        soderj.append(stat.index(i))
    reeng_feach['Keyword1'] = soderj

    stat = {}
    for i in reeng_feach['Keyword2']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    soderj = []
    for i in reeng_feach['Keyword2']:
        soderj.append(stat.index(i))
    reeng_feach['Keyword2'] = soderj

    stat = {}
    for i in reeng_feach['Keyword3']:
        if i not in stat:
            stat[i] = 1
        else:
            stat[i] += 1
    stat = list(stat)
    soderj = []
    for i in reeng_feach['Keyword3']:
        soderj.append(stat.index(i))
    reeng_feach['Keyword3'] = soderj
    return reeng_feach


def relearn():
    CBC = CatBoostClassifier()
    feach = preprocess('Выгрузка 9 апреля.xls')
    feach = feature_eng(feach)
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(feach.drop('Адресат',axis=1), feach['Адресат'], test_size=0.15)
    CBC.fit(x_train,y_train)
    return CBC


if __name__ == "__main__":
    app.run()
